In [99]:
%reset -f
%config InteractiveShell.ast_node_interactivity = 'all'

# Fetching all Air Quality datasets into their dataframes
# Perform immediate concatenation per year
import os
import pandas as pd

# Initialize (do not add extra datasets to dir)
dir = 'CAdata/'
colidx = [0,2,4,17]     # column indexes to use
innerkeys = ['Date', 'Site ID', 'COUNTY']
dataA2020 = pd.read_csv(dir + 'cf-2020-co.csv', parse_dates=True, usecols=colidx)
dataA2021 = pd.read_csv(dir + 'cf-2021-co.csv', parse_dates=True, usecols=colidx)
dataA2022 = pd.read_csv(dir + 'cf-2022-co.csv', parse_dates=True, usecols=colidx)

with os.scandir(dir) as datasets:
    for dataset in datasets:
        if dataset.is_file() and 'co' not in dataset.name:
            temp = pd.read_csv(dataset, parse_dates=True, usecols=colidx)
            if '2020' in dataset.name:
                # cols_to_use = dataA2020.columns.difference(temp)
                dataA2020 = pd.merge(dataA2020, temp, how='outer', on=innerkeys)
            elif '2021' in dataset.name:
                # cols_to_use = dataA2021.columns.difference(temp)
                dataA2021 = pd.merge(dataA2021, temp, how='outer', on=innerkeys)
            elif '2022' in dataset.name:
                # cols_to_use = dataA2022.columns.difference(temp)
                dataA2022 = pd.merge(dataA2022, temp, how='outer', on=innerkeys)

# Parse Dates to date
dataA2020['Date'] = pd.to_datetime(dataA2020['Date'])
dataA2021['Date'] = pd.to_datetime(dataA2020['Date'])
dataA2022['Date'] = pd.to_datetime(dataA2020['Date'])

dataA2020 = dataA2020.groupby(by=['Date', 'Site ID']).mean().groupby(by=['Date']).mean()
dataA2021 = dataA2021.groupby(by=['Date', 'Site ID']).mean().groupby(by=['Date']).mean()
dataA2022 = dataA2022.groupby(by=['Date', 'Site ID']).mean().groupby(by=['Date']).mean()

dataA = pd.concat([dataA2020, dataA2021, dataA2022])

new_names = ['CO conc (ppm)', 'NO2 conc (ppb)', 'O3 conc (ppm)',
             'Pb conc (ug/m3 SC)', 'PM10 conc (ug/m3 SC)',
             'PM2.5 conc (ug/m3 LC)', 'SO2 conc (ppb)']

dataA2020.head()
dataA2021.head()
dataA2022.head()
dataA.head()

# Rename columns
for i in range(len(new_names)):
    dataA.rename(columns={dataA.columns[i]: new_names[i]}, inplace=True)

dataA.head()

import seaborn as sns

# Deciding whether to drop or impute null values, so we check how many null values there are.
# Dataset A impute
print("A: Number of entries with null values:", dataA.isna().any(axis=1).sum())
print("A: Number of entries:", dataA.shape[0])

# These imports are important, imputer relies on them.

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer   # Important!
from sklearn.impute import IterativeImputer     # default imputer is BayesianRidge

from sklearn.linear_model import BayesianRidge

# Initialize imputer
imp = IterativeImputer(max_iter=100, random_state=1, verbose=True)
dataA[:] = imp.fit_transform(dataA)

dataA.head()

,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
2020-01-01,0.606780,19.129570,0.030783,0.008000,17.211765,13.533446,1.236
2020-01-02,0.643220,24.476087,0.028740,0.013000,16.750000,11.087611,1.228
2020-01-03,0.794915,28.616304,0.027643,0.018990,19.861446,13.664307,1.236
2020-01-04,0.817797,24.173913,0.031677,0.011233,19.096774,11.571699,1.140
2020-01-05,0.714407,20.461828,0.033058,0.009000,13.103659,9.578448,0.884


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
2020-01-01,0.421117,17.465129,0.044129,0.008276,30.204377,9.791532,0.996032
2020-01-02,0.398177,17.066677,0.044417,0.009625,30.158784,11.344008,1.093923
2020-01-03,0.393545,18.040451,0.044692,0.011334,30.537281,8.643291,1.715351
2020-01-04,0.407776,19.506657,0.043930,0.009864,27.113144,10.212933,0.920820
2020-01-05,0.408503,19.592124,0.044431,0.008077,30.267311,11.748946,0.867424


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
2020-01-01,0.476892,19.920890,0.043637,0.010526,54.760354,9.117983,1.359524
2020-01-02,0.502391,19.215635,0.043188,0.010061,46.118125,9.194244,1.177981
2020-01-03,0.493259,20.120784,0.043198,0.006170,31.544113,9.105370,0.758787
2020-01-04,0.486350,20.472398,0.043339,0.006181,25.037581,8.565060,0.927782
2020-01-05,0.475717,21.147079,0.043486,0.009130,25.712634,9.122562,0.852417


,Daily Max 8-hour CO Concentration,Daily Max 1-hour NO2 Concentration,Daily Max 8-hour Ozone Concentration,Daily Mean Pb Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 1-hour SO2 Concentration
Date,,,,,,,
2020-01-01,0.606780,19.129570,0.030783,0.008000,17.211765,13.533446,1.236
2020-01-02,0.643220,24.476087,0.028740,0.013000,16.750000,11.087611,1.228
2020-01-03,0.794915,28.616304,0.027643,0.018990,19.861446,13.664307,1.236
2020-01-04,0.817797,24.173913,0.031677,0.011233,19.096774,11.571699,1.140
2020-01-05,0.714407,20.461828,0.033058,0.009000,13.103659,9.578448,0.884


,CO conc (ppm),NO2 conc (ppb),O3 conc (ppm),Pb conc (ug/m3 SC),PM10 conc (ug/m3 SC),PM2.5 conc (ug/m3 LC),SO2 conc (ppb)
Date,,,,,,,
2020-01-01,0.606780,19.129570,0.030783,0.008000,17.211765,13.533446,1.236
2020-01-02,0.643220,24.476087,0.028740,0.013000,16.750000,11.087611,1.228
2020-01-03,0.794915,28.616304,0.027643,0.018990,19.861446,13.664307,1.236
2020-01-04,0.817797,24.173913,0.031677,0.011233,19.096774,11.571699,1.140
2020-01-05,0.714407,20.461828,0.033058,0.009000,13.103659,9.578448,0.884


A: Number of entries with null values: 15
A: Number of entries: 1098
[IterativeImputer] Completing matrix with shape (1098, 7)
[IterativeImputer] Change: 0.009690201551247349, scaled tolerance: 0.2222875 
[IterativeImputer] Early stopping criterion reached.


,CO conc (ppm),NO2 conc (ppb),O3 conc (ppm),Pb conc (ug/m3 SC),PM10 conc (ug/m3 SC),PM2.5 conc (ug/m3 LC),SO2 conc (ppb)
Date,,,,,,,
2020-01-01,0.606780,19.129570,0.030783,0.008000,17.211765,13.533446,1.236
2020-01-02,0.643220,24.476087,0.028740,0.013000,16.750000,11.087611,1.228
2020-01-03,0.794915,28.616304,0.027643,0.018990,19.861446,13.664307,1.236
2020-01-04,0.817797,24.173913,0.031677,0.011233,19.096774,11.571699,1.140
2020-01-05,0.714407,20.461828,0.033058,0.009000,13.103659,9.578448,0.884


In [100]:
colidx = [0,1,2,5,6]     # column indexes to use
dataB = pd.read_csv('datasets/us_covid_cases_and_deaths_by_state.csv', parse_dates=True, usecols=colidx)
sum_new_cases = dataB['new_case'] + dataB['pnew_case']
dataB.drop(['new_case', 'pnew_case'], axis=1, inplace=True)
dataB['sum_new_cases'] = sum_new_cases
dataB.head()

dataB.columns = ['Date', 'State', 'Total Cases', 'Sum New Cases']

# Filter dataset B
dataB = dataB[dataB['State'] == 'CA']
dataB.head()

# Use Date as index, also drop the State
dataB['Date'] = pd.to_datetime(dataB['Date'])
dataB.set_index('Date', inplace=True)
dataB.sort_index(inplace=True)
dataB.drop('State', axis=1, inplace=True)
dataB.info()
dataB.head()

# Filter dataA with temporal restriction given by dataB
dataA = dataA[(dataA.index >= dataB.index.min()) &
              (dataA.index <= dataB.index.max())]

print("Filtered Dataset A")
dataA.head()

# With the printouts below, we find that there's no need to impute.
print("B: Number of entries with null values:", dataB.isna().any(axis=1).sum())
print("B: Number of entries:", dataB.shape[0])

,submission_date,state,tot_cases,sum_new_cases
0,03/11/2021,KS,297229,0.0
1,12/01/2021,ND,163565,809.0
2,01/02/2022,AS,11,0.0
3,11/22/2021,AL,841461,1060.0
4,05/30/2022,AK,251425,0.0


,Date,State,Total Cases,Sum New Cases
15543,06/11/2020,CA,139281,3090.0
15553,08/18/2020,CA,632667,4636.0
15562,02/25/2021,CA,3460326,4965.0
15573,02/10/2021,CA,3362981,8390.0
15576,09/06/2020,CA,732144,4905.0


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1001 entries, 2020-01-22 to 2022-10-18
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Total Cases    1001 non-null   int64  
 1   Sum New Cases  1001 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 23.5 KB


,Total Cases,Sum New Cases
Date,,
2020-01-22,0,0.0
2020-01-23,0,0.0
2020-01-24,0,0.0
2020-01-25,0,0.0
2020-01-26,0,0.0


Filtered Dataset A


,CO conc (ppm),NO2 conc (ppb),O3 conc (ppm),Pb conc (ug/m3 SC),PM10 conc (ug/m3 SC),PM2.5 conc (ug/m3 LC),SO2 conc (ppb)
Date,,,,,,,
2020-01-22,0.525000,24.782447,0.031561,0.008904,13.844353,7.054452,0.743478
2020-01-23,0.670690,26.067553,0.025876,0.010460,15.817647,8.524138,1.087500
2020-01-24,0.690351,25.952660,0.026554,0.015000,17.476190,9.846121,1.052174
2020-01-25,0.585965,21.463402,0.028829,0.009500,18.971591,12.164924,1.433333
2020-01-26,0.508772,17.300521,0.033949,0.007000,16.000000,10.825797,0.954167


B: Number of entries with null values: 0
B: Number of entries: 1001
